## Dependencies
* standard libraries: random, math, itertools
* numpy scipy
* keras / tensorflow

In [2]:
import random
import math
from itertools import permutations
import numpy as np
from scipy.spatial.transform import Rotation as R

In [44]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
K.backend()

'tensorflow'

## Model

In [4]:
all_rotation_orders = [''.join(axes) for axes in permutations({'x','y','z'})]
all_rotation_orders

['xzy', 'xyz', 'zxy', 'zyx', 'yxz', 'yzx']

In [5]:
def random_vector(n, mag):
    return 2.0 * mag * (0.5-np.random.random(size=n))
random_vector(10,5.0)

array([ 2.7369162 , -0.92583108,  4.12866617, -1.01455891,  4.00566382,
        4.72884498,  2.38688827, -2.02173267,  2.01349269,  4.44800766])

define a function to generate a single sample from the model.

sample is a tuple of:
* np.array with 9 matrix elements + 3 rotation angles
* string corresponding to chosen rotation order (element of all_rotation_orders)

In [6]:
def generate_sample_from_model():
    rotation_order = random.choice(all_rotation_orders)  # choose a rotation order
    angles = random_vector(3, 10.0)   # generate three random angles (in degrees)
    if (min(abs(angles)) < 0.1):     # reject any angles that are too small
        return generate_sample_from_model()
    matrix = R.from_euler(rotation_order, angles, degrees = True)  # generate matrix
    noise_angles = angles + random_vector(3, 0.1)    
    x = np.concatenate((matrix.as_dcm().flatten(), 
                        np.array(noise_angles)))
                        # np.array(np.cos(noise_angles)), 
                        # np.array(np.sin(noise_angles))))                         
    return (x, rotation_order)
generate_sample_from_model()

(array([ 0.99888836, -0.01339326, -0.0451958 ,  0.01755009,  0.99552368,
         0.09286873,  0.04374968, -0.09355869,  0.99465207, -2.61468866,
         0.78302637, -5.46866772]), 'yzx')

define a function to generate multiple samples, suitable for training or testing.

for number N, sample is a tuple:
 * np.array Nx12 of x vectors (standardized zero mean)
 * np.array Nx6 one-hot vectors

In [7]:
def generate_samples(n):
    [xs, rotation_orders] = list(zip(*[generate_sample_from_model() for _ in range(n)]))
    xs = np.array(xs)
    xs -= xs.mean(0)    # standardize along sample dimension
    xs /= xs.std(0)
    ys = [all_rotation_orders.index(rotation_order) for rotation_order in rotation_orders]
    return xs, keras.utils.to_categorical(ys)
generate_samples(3)

(array([[-0.38552229,  0.64717839, -1.41420869, -0.68065051,  0.85828697,
         -0.06216144,  1.39230228,  0.03451528, -0.04341979, -0.84040034,
         -0.52015687,  1.04567727],
        [-0.98559764, -1.41256582,  0.7038904 ,  1.41388762, -1.40254351,
         -1.19248046, -0.4813937 ,  1.20712241, -1.20245759,  1.40523593,
          1.39897159, -1.34741082],
        [ 1.37111993,  0.76538744,  0.71031829, -0.73323712,  0.54425654,
          1.2546419 , -0.91090859, -1.2416377 ,  1.24587739, -0.56483559,
         -0.87881472,  0.30173355]]), array([[0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]], dtype=float32))

generate training and testing data from the model

In [8]:
x_train, y_train = generate_samples(10000)
x_test, y_test = generate_samples(1000)
x_train.shape, y_train.shape

((10000, 12), (10000, 6))

now create an MLP as the model

In [9]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = x_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 4,166
Trainable params: 4,166
Non-trainable params: 0
_________________________________________________________________


set up the optimizer and compile the model

In [10]:
adam = Adam(lr=0.001, decay=1e-8)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

now fit the data using the training samples

In [11]:
model.fit(x_train, y_train, epochs=200, batch_size=256)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
10000/10000 [==============================] - 0s 26us/step - loss: 1.7222 - acc: 0.2790
Epoch 2/200
10000/10000 [==============================] - 0s 6us/step - loss: 1.4826 - acc: 0.4321
Epoch 3/200
10000/10000 [==============================] - 0s 5us/step - loss: 1.1311 - acc: 0.5861
Epoch 4/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.8076 - acc: 0.7376
Epoch 5/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.6003 - acc: 0.8063
Epoch 6/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.4731 - acc: 0.8656
Epoch 7/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.4015 - acc: 0.8780
Epoch 8/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.3440 - acc: 0.9029
Epoch 9/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.3122 - acc: 0.9048
Epoch 10/200
10000/10000 [=========================

10000/10000 [==============================] - 0s 6us/step - loss: 0.0421 - acc: 0.9911
Epoch 80/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0420 - acc: 0.9900
Epoch 81/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0479 - acc: 0.9862
Epoch 82/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0416 - acc: 0.9890
Epoch 83/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0439 - acc: 0.9882
Epoch 84/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0425 - acc: 0.9880
Epoch 85/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0477 - acc: 0.9854
Epoch 86/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0432 - acc: 0.9887
Epoch 87/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0436 - acc: 0.9879
Epoch 88/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0409 - acc: 0.9894
Epo

10000/10000 [==============================] - 0s 5us/step - loss: 0.0251 - acc: 0.9921
Epoch 160/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0333 - acc: 0.9893
Epoch 161/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0600 - acc: 0.9839
Epoch 162/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0370 - acc: 0.9865
Epoch 163/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0213 - acc: 0.9959
Epoch 164/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0228 - acc: 0.9942
Epoch 165/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0237 - acc: 0.9927
Epoch 166/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0248 - acc: 0.9924
Epoch 167/200
10000/10000 [==============================] - 0s 5us/step - loss: 0.0302 - acc: 0.9915
Epoch 168/200
10000/10000 [==============================] - 0s 6us/step - loss: 0.0200 - acc: 0

In [12]:
score = model.evaluate(x_test, y_test, batch_size=256)
print(score)

1000/1000 [==============================] - 0s 39us/step
[0.052335924088954926, 0.9740000014305115]


In [18]:
model.save('rotation_order.h5')

In [21]:
import h5py
f = h5py.File('rotation_order.h5')

In [22]:
f

<HDF5 file "rotation_order.h5" (mode r+)>

In [36]:
[(k, grp) for k in f.keys() for grp in f[k]]

[('model_weights', 'dense_1'),
 ('model_weights', 'dense_2'),
 ('model_weights', 'dense_3'),
 ('model_weights', 'dense_4'),
 ('optimizer_weights', 'Adam'),
 ('optimizer_weights', 'training')]

In [37]:
type(f)

h5py._hl.files.File

In [53]:
import pprint
f.visititems(lambda name, obj: pprint.pprint(obj))
# f['model_weights']['dense_1'].name

<HDF5 group "/model_weights" (4 members)>
<HDF5 group "/model_weights/dense_1" (1 members)>
<HDF5 group "/model_weights/dense_1/dense_1" (2 members)>
<HDF5 dataset "bias:0": shape (64,), type "<f4">
<HDF5 dataset "kernel:0": shape (12, 64), type "<f4">
<HDF5 group "/model_weights/dense_2" (1 members)>
<HDF5 group "/model_weights/dense_2/dense_2" (2 members)>
<HDF5 dataset "bias:0": shape (32,), type "<f4">
<HDF5 dataset "kernel:0": shape (64, 32), type "<f4">
<HDF5 group "/model_weights/dense_3" (1 members)>
<HDF5 group "/model_weights/dense_3/dense_3" (2 members)>
<HDF5 dataset "bias:0": shape (32,), type "<f4">
<HDF5 dataset "kernel:0": shape (32, 32), type "<f4">
<HDF5 group "/model_weights/dense_4" (1 members)>
<HDF5 group "/model_weights/dense_4/dense_4" (2 members)>
<HDF5 dataset "bias:0": shape (6,), type "<f4">
<HDF5 dataset "kernel:0": shape (32, 6), type "<f4">
<HDF5 group "/optimizer_weights" (2 members)>
<HDF5 group "/optimizer_weights/Adam" (1 members)>
<HDF5 dataset "iter

In [43]:
model.get_config()

{'name': 'sequential_1',
 'layers': [{'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'batch_input_shape': (None, 12),
    'dtype': 'float32',
    'units': 64,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_2',
    'trainable': True,
    'units': 32,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', '

In [46]:
[(layer.name, weight_variable.name, weight_values)
     for layer in model.layers 
     for (weight_variable, weight_values) 
     in zip(layer.weights, K.batch_get_value(layer.weights))]

[('dense_1',
  'dense_1/kernel:0',
  array([[ 5.09666055e-02,  1.79672502e-02, -2.01220834e-03,
           2.03947812e-01,  5.94712012e-02,  6.65819496e-02,
          -2.72270422e-02,  7.00616464e-03, -2.29848996e-02,
           2.24079713e-02,  1.15321875e-01,  1.14370205e-01,
          -2.46004406e-02, -1.11476719e-01,  6.19212873e-02,
           2.98484147e-01, -1.65693220e-02, -2.78346911e-02,
          -3.06334626e-02, -1.98525235e-01, -5.45634925e-02,
          -6.28091097e-02, -7.05258772e-02,  2.29806900e-01,
          -5.74674532e-02,  1.13768911e-03, -4.44827564e-02,
           5.55592962e-02,  3.46210063e-03,  1.94255874e-01,
          -2.59008482e-02,  9.01120976e-02, -1.94241037e-03,
           1.15775675e-01, -3.86727452e-02, -2.59493917e-01,
           2.70995796e-01,  8.71656463e-03, -5.99553958e-02,
          -1.80835109e-02, -4.24945615e-02, -2.56717484e-02,
          -3.91839519e-02,  1.11980550e-02, -3.27764824e-02,
          -1.36963278e-01,  5.36150951e-03, -1.781

In [88]:
def row_n(values):
    if len(values.shape) == 1:
        return 1
    else:
        return values.shape[0]
    
def get_row(values, n):
    if len(values.shape) == 1:
        return values
    else:
        return values[n]
    
[(layer.name, weight_variable.name, weight_values.shape, row_num, get_row(weight_values, row_num))
     for layer in model.layers 
     for (weight_variable, weight_values) in zip(layer.weights, K.batch_get_value(layer.weights)) 
     for row_num in range(row_n(weight_values))]

[('dense_1',
  'dense_1/kernel:0',
  (12, 64),
  0,
  array([ 0.05096661,  0.01796725, -0.00201221,  0.20394781,  0.0594712 ,
          0.06658195, -0.02722704,  0.00700616, -0.0229849 ,  0.02240797,
          0.11532187,  0.1143702 , -0.02460044, -0.11147672,  0.06192129,
          0.29848415, -0.01656932, -0.02783469, -0.03063346, -0.19852524,
         -0.05456349, -0.06280911, -0.07052588,  0.2298069 , -0.05746745,
          0.00113769, -0.04448276,  0.0555593 ,  0.0034621 ,  0.19425587,
         -0.02590085,  0.0901121 , -0.00194241,  0.11577567, -0.03867275,
         -0.25949392,  0.2709958 ,  0.00871656, -0.0599554 , -0.01808351,
         -0.04249456, -0.02567175, -0.03918395,  0.01119805, -0.03277648,
         -0.13696328,  0.00536151, -0.17814735, -0.0867773 ,  0.06027253,
          0.08252747,  0.06106157,  0.02237234,  0.06475427, -0.03642348,
          0.037214  ,  0.03955822,  0.16373909,  0.10348999, -0.0031756 ,
         -0.03068971, -0.27688596,  0.04962901,  0.05228548]

In [78]:
import csv
with open('eggs.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
    spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])
    spamwriter.writerow(np.array([1,2,3,4]))

In [92]:
with open('weights.csv', 'w', newline='') as csvfile:
    weightswriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    for layer in model.layers:
        for weight_variable in layer.weights:
            weight_values = K.batch_get_value(weight_variable)
            for row_num in range(row_n(weight_values)):
                row = [layer.name, weight_variable.name, weight_values.shape, row_num] + list(get_row(weight_values, row_num))
                weightswriter.writerow(row)